# Gaps and Islands

Objective: convert start and end date spans of overlapping dates to unique non-overlapping groups.

The key is to identify the start of new islands, assign an island/group ID, then take the min() and max() of start and end dates, respectively.

From
```
|   row_num | start      | end        | prev_end   |   island_start |   island_id |
|----------:|:-----------|:-----------|:-----------|---------------:|------------:|
|         0 | 2017-08-24 | 2017-09-20 | NaT        |              1 |           1 |
|         1 | 2017-08-24 | 2017-09-23 | 2017-09-20 |              0 |           1 |
|         3 | 2017-09-23 | 2017-09-26 | 2017-09-23 |              0 |           1 |
|         2 | 2017-09-23 | 2017-09-27 | 2017-09-26 |              0 |           1 |
|         4 | 2017-09-25 | 2017-10-10 | 2017-09-27 |              0 |           1 |
|         5 | 2017-10-17 | 2017-10-18 | 2017-10-10 |              1 |           2 |
|         6 | 2017-10-25 | 2017-11-03 | 2017-10-18 |              1 |           3 |
|         7 | 2017-11-03 | 2017-11-15 | 2017-11-03 |              0 |           3 |
```

To
```
|   island_id | start      | end        |
|------------:|:-----------|:-----------|
|           1 | 2017-08-24 | 2017-10-10 |
|           2 | 2017-10-17 | 2017-10-18 |
|           3 | 2017-10-25 | 2017-11-15 |
```

# Gaps and Islands in Python

In [1]:
import pandas as pd
import numpy as np

df0 = pd.DataFrame([['2017-08-24', '2017-09-20'],
 ['2017-08-24', '2017-09-23'],
 ['2017-09-23', '2017-09-27'],
 ['2017-09-23', '2017-09-26'],
 ['2017-09-25', '2017-10-10'],
 ['2017-10-17', '2017-10-18'],
 ['2017-10-25', '2017-11-03'],
 ['2017-11-03', '2017-11-15']], columns=['start', 'end'], dtype=('datetime64[ns]', 'datetime64[ns]'))

df = df0.copy()
df = df.sort_values(by=['start', 'end'])
df.index = df.index.rename('row_num')
df['prev_end'] = df['end'].shift()
island_bool = df['prev_end'] >= df['start']
df['island_start'] = (~island_bool).astype(int)
df['island_id'] = df.groupby(['island_start'])['island_start'].cumsum()
df['island_id'] = df['island_id'].replace(0, np.NaN).ffill().astype(int)
islands_grouped = df.groupby(['island_id']).agg({'start': 'min', 'end': 'max'})

In [2]:
df

,start,end,prev_end,island_start,island_id
row_num,,,,,
0,2017-08-24,2017-09-20,NaT,1,1
1,2017-08-24,2017-09-23,2017-09-20,0,1
3,2017-09-23,2017-09-26,2017-09-23,0,1
2,2017-09-23,2017-09-27,2017-09-26,0,1
4,2017-09-25,2017-10-10,2017-09-27,0,1
5,2017-10-17,2017-10-18,2017-10-10,1,2
6,2017-10-25,2017-11-03,2017-10-18,1,3
7,2017-11-03,2017-11-15,2017-11-03,0,3


In [3]:
islands_grouped

,start,end
island_id,,
1,2017-08-24,2017-10-10
2,2017-10-17,2017-10-18
3,2017-10-25,2017-11-15


# Gaps and Islands in SQL

Source: https://bertwagner.com/2019/03/12/gaps-and-islands/

```sql
DROP TABLE IF EXISTS #OverlappingDateRanges;
CREATE TABLE #OverlappingDateRanges (StartDate date, EndDate date);

INSERT INTO #OverlappingDateRanges
VALUES 
('8/24/2017', '9/23/2017')
,('8/24/2017', '9/23/2017')
,('8/24/2017', '9/23/2017') 
,('8/24/2017', '9/20/2017')  
,('9/23/2017', '9/27/2017')  
,('9/25/2017', '10/10/2017')
,('10/17/2017','10/18/2017') 
,('10/25/2017','11/3/2017')  
,('11/3/2017', '11/15/2017');

SELECT * FROM #OverlappingDateRanges

SELECT
    MIN(StartDate) AS IslandStartDate,
    MAX(EndDate) AS IslandEndDate
FROM
    (
    SELECT
        *,
        CASE WHEN Groups.PreviousEndDate >= StartDate THEN 0 ELSE 1 END AS IslandStartInd,
        SUM(CASE WHEN Groups.PreviousEndDate >= StartDate THEN 0 ELSE 1 END) OVER (ORDER BY Groups.RN) AS IslandId
    FROM
    (
        SELECT
            ROW_NUMBER() OVER(ORDER BY StartDate,EndDate) AS RN,
            StartDate,
            EndDate,
            LAG(EndDate,1) OVER (ORDER BY StartDate, EndDate) AS PreviousEndDate
        FROM
            #OverlappingDateRanges
    ) Groups
) Islands
GROUP BY
    IslandId
ORDER BY 
    IslandStartDate

```

Draft
# Testing for Continuity in Dates
Identifying the number of dates greater than allowable threshold of `n` days

In [4]:
# pd.read_clipboard().values.tolist()

In [5]:
from datetime import datetime

df = pd.DataFrame([['2018-01-01', '2018-02-28'],
 ['2018-08-01', '2018-09-30'],
 ['2018-10-01', '2019-01-05'],
 ['2019-01-06', '2019-01-31'],
 ['2019-02-01', '2019-02-28'],
 ['2019-03-01', '2019-03-31'],
 ['2019-05-01', '2019-05-15'],
 ['2019-06-01', '2019-07-02'],
 ['2019-06-15', '2019-07-30'],
 ['2019-08-01', '9999-12-31']], columns=['start', 'end'])


def convert_hi_date(row):
    '''If high-date, then assume term at end of the start year.'''
    
    if row['end'] != '9999-12-31':
        return row['end']
    else:
        start_year = row['start'][:4]
        default_end_year = start_year
        default_end_date = f'{default_end_year}-12-31'
        return default_end_date
    
    
def clean_up(df):
    df = df.sort_values(by=['start', 'end'])
    df.index = df.index.rename('row_num')
    df['prev_end'] = df['end'].shift()
    island_bool = df['prev_end'] + pd.Timedelta(days=1) >= df['start']
    df['island_start'] = (~island_bool).astype(int)
    df['island_id'] = df.groupby(['island_start'])['island_start'].cumsum()
    df['island_id'] = df['island_id'].replace(0, np.NaN).ffill().astype(int)
    islands_grouped = df.groupby(['island_id']).agg({'start': 'min', 'end': 'max'})
    return islands_grouped

# TODO: add gap check for later part of year
# ex: 1/1 - 4/30 enrollment is a gap for 5/1-12/31
def gaps_df(islands_grouped, my_start=None, my_end=None, n_days=15):
    '''Filters to islands in measurement period and determines # and magnitude of gaps.
    
    Arguments
    ---------
    islands_grouped (pd.DataFrame)
    my_start (datetime.datetime)
    my_end (datetime.datetime)
    n_days (int)

    '''

    # check if dates are in measurement period and filter down
    _1 = islands_grouped['start'].between(my_start, my_end, inclusive=True)
    _2 = islands_grouped['end'].between(my_start, my_end, inclusive=True)
    in_my_bool = (_1 | _2)
    islands_grouped['in_my'] = in_my_bool.astype(int)
    islands_filtered = islands_grouped[in_my_bool].copy()

    # old island ID #s do not apply
    del islands_grouped    
    islands_filtered = islands_filtered.reset_index(drop=True)

    
    islands_filtered['prev_end'] = islands_filtered['end'].shift()

    # for first record only, set prev_end to beginning of measure year
    # this will help determine gap in early part of year
    # ex: enrollment start on 1/5/19 of 1/1/19 measure year start
    islands_filtered.loc[0, 'prev_end'] = my_start
    islands_filtered['zero'] = pd.Timedelta(0, unit='d')
    islands_filtered['diff'] = islands_filtered['start'] - islands_filtered['prev_end']

    islands_filtered['gap'] = (islands_filtered['diff'] >= islands_filtered['zero']).astype(int)
    
    n_days = 15
    
    # gaps greater than threashold
    islands_filtered['gaps_gt_thresh'] = (islands_filtered['diff'] > pd.Timedelta(n_days, unit='d')).astype(int)
    
    return islands_filtered

In [6]:
my_start = datetime.strptime('2019-01-01', '%Y-%m-%d')
my_end = datetime.strptime('2019-12-31', '%Y-%m-%d')

df['end'] = df.apply(lambda x: convert_hi_date(x), axis=1)
df[['start', 'end']] = df[['start', 'end']].apply(pd.to_datetime)
islands_grouped = clean_up(df)
islands_filtered = gaps_df(islands_grouped, my_start, my_end, n_days=15)

In [7]:
islands_filtered

,start,end,in_my,prev_end,zero,diff,gap,gaps_gt_thresh
0,2018-08-01,2019-03-31,1,2019-01-01,0 days,-153 days,0,0
1,2019-05-01,2019-05-15,1,2019-03-31,0 days,31 days,1,1
2,2019-06-01,2019-07-30,1,2019-05-15,0 days,17 days,1,1
3,2019-08-01,2019-12-31,1,2019-07-30,0 days,2 days,1,0


In [8]:
islands_filtered['gaps_gt_thresh'].sum()

2